In [1]:
import chess
from stockfish import Stockfish
from ollama import chat
from ollama import ChatResponse
import json
import random
import pandas as pd

In [2]:
# STOCKFISH_PATH = r"C:\Users\nafis\Downloads\stockfish-windows-x86-64-avx2\stockfish\stockfish-windows-x86-64-avx2.exe"

STOCKFISH_ELO = 3500
NUM_GAMES = 60
MOVE_LIMIT = 99

In [3]:
stockfish = Stockfish()


In [4]:
qwen_white_wins = 0
qwen_black_wins = 0
draws = 0

In [5]:
# board2 = chess.Board()

In [6]:
def valid_move_to_string(board):
    legal_moves = [
        board.san(move) for move in board.legal_moves]
    
    if not legal_moves:
        return "No legal moves."
    else:
        return ", ".join(legal_moves)

In [7]:
# board2.legal_moves

# valid_move_to_string(board2)

In [8]:
def get_move_history(board: chess.Board) -> str:
    """Return a formatted move history string like '1. e4 e5 2. Nf3 d5'."""
    temp_board = chess.Board()  # Start from the initial position.
    move_history = []
    
    # Iterate over the moves in the board's move stack.
    for i, move in enumerate(board.move_stack):
        # Get the SAN for the move in the current temporary board.
        move_san = temp_board.san(move)
        # Push the move so that the board state is updated.
        temp_board.push(move)
        
        # For white moves (even index), start a new entry with move number.
        if i % 2 == 0:
            move_history.append(f"{i//2 + 1}. {move_san}")
        # For black moves, append to the last white move entry.
        else:
            move_history[-1] += f" {move_san}"
    
    # Join all entries with a space.
    return " ".join(move_history)

In [9]:
def get_qwen_move(board:chess.Board, fen:str, temp=0.0):
    valid_moves = valid_move_to_string(board)

    move_history = get_move_history(board)
    
    response: ChatResponse = chat(model='qwen:32b', messages=[
        {
            'role': 'system',
            'content': 'You are a grand master chess player.'
        },
        {
            'role': 'user',
            'content': f"""You are playing chess and it is your turn. This is the current state of the game. Use this to work out where the pieces are on the board:

FEN: {fen}

The possible set of legal moves are: 

Legal Moves: {valid_moves}

+(You have to choose one from the provided list. Do not choose a move that is not in the list.)

The move history is: {move_history}.

Output the best move in SAN format to follow this position. Use the following single blob of JSON. Do not include any other information.
{{
    "san": "The move in SAN format",
}}"""}], options={"temperature": temp})
    
    try:
        parsed_json = json.loads(response.message.content)
        qwen_move = parsed_json["san"]
    except Exception as e:
        print(f"Wrong format given by Qwen. \nQwen Output: {response.message.content}")

    

    return qwen_move

In [10]:
def get_random_move(board:chess.Board, fen:str, seed: int = 101):
    legal_moves = [
        board.san(move)
        for move in board.legal_moves
    ]

    random.seed(seed)

    # print(random.choice(legal_moves))
    return random.choice(legal_moves)

    # print(legal_moves[0])

In [ ]:
for game_number in range(1, NUM_GAMES + 1):
    
    move_counter = 0

    print(f"Starting game {game_number}...")

    board = chess.Board()

    while not board.is_game_over() and move_counter <= MOVE_LIMIT:
        if board.turn == chess.WHITE and not board.is_game_over():
            qwen_move_white = get_qwen_move(board, board.fen(), 2.0)

            try:
                board.push_san(qwen_move_white)
            except Exception as e:
                print(f"Invalid move by Qwen white. Move: {qwen_move_white}")
                qwen_move_white = get_random_move(board, board.fen(), 42)
                board.push_san(qwen_move_white)
                print(f"New Move: {qwen_move_white}")
                # continue
            
            move_counter += 1        
            print(f"Qwen white move: {qwen_move_white} Move number: {move_counter}")

        if board.turn == chess.BLACK and not board.is_game_over():
            qwen_move_black = get_qwen_move(board, board.fen(), 2.0)

            try:
                board.push_san(qwen_move_black)
            except Exception as e:
                print(f"Invalid move by Qwen black. Move: {qwen_move_black}")
                qwen_move_black = get_random_move(board, board.fen(), 42)
                board.push_san(qwen_move_black)
                print(f"New Move: {qwen_move_black}")
                # continue
            move_counter += 1
            print(f"Qwen black move: {qwen_move_black} Move number: {move_counter}")   

    # Record the result of the game
    result = board.result()
    if result == "1-0":
        qwen_white_wins += 1
    elif result == "0-1":
        qwen_black_wins += 1
    else:
        draws += 1

    print(f"Game {game_number} result: {result}\n\n")

Starting game 1...
Qwen white move: e4 Move number: 1
Qwen black move: e5 Move number: 2
Qwen white move: Nf3 Move number: 3
Qwen black move: Nc6 Move number: 4
Qwen white move: Ng5 Move number: 5
Qwen black move: Nh6 Move number: 6
Invalid move by Qwen white. Move: Nxe5
New Move: Bb5
Qwen white move: Bb5 Move number: 7
Invalid move by Qwen black. Move: Bxc6
New Move: f6
Qwen black move: f6 Move number: 8
Qwen white move: Bxc6 Move number: 9
Invalid move by Qwen black. Move: Bxc6
New Move: b5
Qwen black move: b5 Move number: 10
Qwen white move: Bxd7+ Move number: 11
Qwen black move: Bxd7 Move number: 12
Qwen white move: Nh3 Move number: 13
Qwen black move: Bd6 Move number: 14
Qwen white move: Nf4 Move number: 15
Invalid move by Qwen black. Move: Nxe5
New Move: Qe7
Qwen black move: Qe7 Move number: 16
Qwen white move: Nh3 Move number: 17
Qwen black move: Ng4 Move number: 18
Qwen white move: Ng5 Move number: 19
Invalid move by Qwen black. Move: Nf4
New Move: Qf8
Qwen black move: Qf8 Move

In [12]:
board.result()

'*'

In [13]:
results = []

In [14]:
results.append({
            "Qwen White Wins": qwen_white_wins,
            "Qwen Black Wins": qwen_black_wins,
            "Draws": draws,
        })

print(results)

[{'Qwen White Wins': 3, 'Qwen Black Wins': 3, 'Draws': 54}]


In [15]:
df = pd.DataFrame(results)
df.to_csv("Qwen vs Qwen.csv", index=False)
print("Results saved to Qwen vs Qwen.csv")

Results saved to Qwen vs Qwen.csv
